<a href="https://colab.research.google.com/github/dkraynev/Tasks/blob/main/321_User_Activity_for_the_Past_30_Days_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Table: Activity

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user_id       | int     |
| session_id    | int     |
| activity_date | date    |
| activity_type | enum    |
+---------------+---------+
This table may have duplicate rows.
The activity_type column is an ENUM (category) of type ('open_session', 'end_session', 'scroll_down', 'send_message').
The table shows the user activities for a social media website.
Note that each session belongs to exactly one user.


Write a solution to find the daily active user count for a period of 30 days ending 2019-07-27 inclusively. A user was active on someday if they made at least one activity on that day.

Return the result table in any order.

The result format is in the following example.



Example 1:

Input:
Activity table:
+---------+------------+---------------+---------------+
| user_id | session_id | activity_date | activity_type |
+---------+------------+---------------+---------------+
| 1       | 1          | 2019-07-20    | open_session  |
| 1       | 1          | 2019-07-20    | scroll_down   |
| 1       | 1          | 2019-07-20    | end_session   |
| 2       | 4          | 2019-07-20    | open_session  |
| 2       | 4          | 2019-07-21    | send_message  |
| 2       | 4          | 2019-07-21    | end_session   |
| 3       | 2          | 2019-07-21    | open_session  |
| 3       | 2          | 2019-07-21    | send_message  |
| 3       | 2          | 2019-07-21    | end_session   |
| 4       | 3          | 2019-06-25    | open_session  |
| 4       | 3          | 2019-06-25    | end_session   |
+---------+------------+---------------+---------------+
Output:
+------------+--------------+
| day        | active_users |
+------------+--------------+
| 2019-07-20 | 2            |
| 2019-07-21 | 2            |
+------------+--------------+
Explanation: Note that we do not care about days with zero active users.

In [1]:
import pandas as pd

def user_activity(activity: pd.DataFrame) -> pd.DataFrame:
    # Фильтруем данные, оставляя только последние 30 дней до и включая 2019-07-27
    filtered_activity = activity[
        (activity['activity_date'] >= '2019-06-28') & (activity['activity_date'] <= '2019-07-27')
    ]

    # Убираем дубликаты по колонкам user_id и activity_date, чтобы считать только уникальных пользователей на каждый день
    unique_users_per_day = filtered_activity[['user_id', 'activity_date']].drop_duplicates()

    # Группируем по дате активности и считаем уникальных пользователей на каждый день
    daily_active_users = unique_users_per_day.groupby('activity_date').agg(active_users=('user_id', 'nunique')).reset_index()

    # Переименовываем колонку 'activity_date' в 'day' для соответствия формату вывода
    daily_active_users.rename(columns={'activity_date': 'day'}, inplace=True)

    return daily_active_users

In [2]:
# Создаем тестовый DataFrame с данными
data = {
    'user_id': [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4],
    'session_id': [1, 1, 1, 4, 4, 4, 2, 2, 2, 3, 3],
    'activity_date': [
        '2019-07-20', '2019-07-20', '2019-07-20',
        '2019-07-20', '2019-07-21', '2019-07-21',
        '2019-07-21', '2019-07-21', '2019-07-21',
        '2019-06-25', '2019-06-25'
    ],
    'activity_type': [
        'open_session', 'scroll_down', 'end_session',
        'open_session', 'send_message', 'end_session',
        'open_session', 'send_message', 'end_session',
        'open_session', 'end_session'
    ]
}

# Преобразуем данные в DataFrame
test_activity_df = pd.DataFrame(data)

# Преобразуем столбец activity_date в тип datetime
test_activity_df['activity_date'] = pd.to_datetime(test_activity_df['activity_date'])

# Вызываем функцию для тестирования
result = user_activity(test_activity_df)

# Выводим результат для проверки
print(result)


         day  active_users
0 2019-07-20             2
1 2019-07-21             2
